In [2]:
import numpy as np
import sympy as sym
import pydae.build as db
from pydae.grid_b_pu import bal_pu

In [11]:
def gformer_add(grid):
    sin = sym.sin
    cos = sym.cos
    buses = grid['data']['buses']
    buses_list = [bus['name'] for bus in buses]
    for gformer in grid['data']['gformers']:

        bus_name = gformer['bus']
        idx_bus = buses_list.index(bus_name) # get the number of the bus where the syn is connected
        if not 'idx_powers' in buses[idx_bus]: buses[idx_bus].update({'idx_powers':0})
        buses[idx_bus]['idx_powers'] += 1

        p_g = sym.Symbol(f"p_g_{bus_name}_{buses[idx_bus]['idx_powers']}", real=True)
        q_g = sym.Symbol(f"q_g_{bus_name}_{buses[idx_bus]['idx_powers']}", real=True)
        V = sym.Symbol(f"V_{bus_name}", real=True)
        theta = sym.Symbol(f"theta_{bus_name}", real=True)
        i_d = sym.Symbol(f"i_d_{bus_name}", real=True)
        i_q = sym.Symbol(f"i_q_{bus_name}", real=True)
        delta = sym.Symbol(f"delta_{bus_name}", real=True)
        omega = sym.Symbol(f"omega_{bus_name}", real=True)
        p_ref = sym.Symbol(f"p_ref_{bus_name}", real=True)
        q_ref = sym.Symbol(f"q_ref_{bus_name}", real=True)
        e = sym.Symbol(f"e_{bus_name}", real=True)
        
        omega_coi = sym.Symbol("omega_coi", real=True)

        p_m_ref  = sym.Symbol(f"p_m_ref_{bus_name}", real=True)
        v_ref  = sym.Symbol(f"v_ref_{bus_name}", real=True)
        xi_m = sym.Symbol(f"xi_m_{bus_name}", real=True)
        
        v_d = V*sin(delta - theta) 
        v_q = V*cos(delta - theta) 

        for item in ['S_n','Omega_b','T_omega','T_e','X_v','R_v',
                     'K_wp','K_wq','K_ep','K_eq',
                      'K_delta']:
            string = f"{item}=sym.Symbol('{item}_{bus_name}', real = True)" 
            exec(string,globals())
            grid['params'].update({f'{item}_{bus_name}':gformer[item]})

        p_e = i_d*(v_d + R_v*i_d) + i_q*(v_q + R_v*i_q) 

        omega_s = omega_coi
        
        omega_ref = 1 + K_wp*(p_ref - p_g) + K_wq*(q_ref - q_g)
        e_ref     = 1 + K_ep*(p_ref - p_g) + K_eq*(q_ref - q_g)
        
        ddelta = Omega_b*(omega - omega_s) - K_delta*delta
        domega = 1/T_omega*(omega_ref - omega)
        de     = 1/T_e*(e_ref - e)
       
        g_i_d  = v_q + R_v*i_q + X_v*i_d - e
        g_i_q  = v_d + R_v*i_d - X_v*i_q
        g_p_g  = i_d*v_d + i_q*v_q - p_g  
        g_q_g  = i_d*v_q - i_q*v_d - q_g 
                
        if 'f' not in grid: grid.update({'f':[]})
        if 'x' not in grid: grid.update({'x':[]})
        grid['f'] += [ddelta,domega,de]
        grid['x'] += [ delta, omega, e]
        grid['g'] += [g_i_d,g_i_q,g_p_g,g_q_g]
        grid['y'] += [i_d,i_q,p_g,q_g]  
        
        S_base = sym.Symbol('S_base', real = True)
        grid['g'][idx_bus*2]   += -p_g*S_n/S_base
        grid['g'][idx_bus*2+1] += -q_g*S_n/S_base

        grid['u'].update({f'p_ref_{bus_name}':0.0})
        grid['u'].update({f'q_ref_{bus_name}':0.0})
        #for item in syn:       
        #    grid['params'].update({f"{item}_{bus_name}":syn[item]})
        #grid['params'].pop(f"bus_{bus_name}")
        #

In [12]:
grid = bal_pu('gform_3bus.json') 

In [13]:
gformer_add(grid)

In [14]:
params_dict = grid['params']

g_list = grid['g']
h_dict = grid['h']
f_list = grid['f']
x_list = grid['x']

sys = {'name':'gform_3bus',
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':grid['y'],
       'y_run_list':grid['y'],
       'u_run_dict':grid['u'],
       'u_ini_dict':grid['u'],
       'h_dict':h_dict}

sys = db.system(sys)
db.sys2num(sys)